# The ***GroupBy*** Pattern

We have seen the ***GroupBy*** operator in ***Pandas***, but  this is actually a more general ***design pattern*** that can be utilized in many data analyics frameworks and data access interfaces, e.g. in ***SQL***. 

## GroupBy: general Pattern
<img SRC="IMG/groupby.jpg">

#### GroupBy in SQL:
```
SELECT COUNT(CustomerID), Country
FROM Customers
GROUP BY Country
ORDER BY COUNT(CustomerID) DESC;
```

#### GroupBy in MongoDB
```
db.BusinessProcess.aggregate({
    "$group": {
        _id: {
            status: "$status",
            type: "$type"
        },
        count: {
            $sum: 1
        }
    }
   })
```


In [1]:
#setup example
import numpy as np
import pandas as pd
df = pd.DataFrame({'key1' : ['a', 'a', 'b', 'b', 'a'],
                   'key2' : ['one', 'two', 'one', 'two', 'one'],
                   'data1' : np.random.randn(5),
                   'data2' : np.random.randn(5)})
df

,key1,key2,data1,data2
0,a,one,-0.317571,-0.946989
1,a,two,-0.315828,-0.649236
2,b,one,0.595599,-1.119620
3,b,two,0.715355,-0.160160
4,a,one,-0.934310,-0.712609


In [2]:
#group by key1 
grouped = df.groupby(df['key1'])
grouped #this is now a more complex group object

In [3]:
#group by creates a group for every unique key
df['key1'].unique()

array(['a', 'b'], dtype=object)

In [4]:
#and generates a table per group
for name, group in grouped:
    print ("name:", name, "\n",group)

name: a 
   key1 key2     data1     data2
0    a  one -0.317571 -0.946989
1    a  two -0.315828 -0.649236
4    a  one -0.934310 -0.712609
name: b 
   key1 key2     data1    data2
2    b  one  0.595599 -1.11962
3    b  two  0.715355 -0.16016


In [6]:
#access group table
grouped.get_group('a')

,key1,key2,data1,data2
0,a,one,-0.317571,-0.946989
1,a,two,-0.315828,-0.649236
4,a,one,-0.934310,-0.712609


In [8]:
#get numper of entries (rows) per group
grouped.size()

key1
a    3
b    2
dtype: int64

In [9]:
#get number of group entries by columns
grouped.count()

,key2,data1,data2
key1,,,
a,3,3,3
b,2,2,2


#### Think of  grouped DataFrames as 3d objects:

In [10]:
#accessing the "3d" group tables
grouped['data2'].get_group('a')

0   -0.946989
1   -0.649236
4   -0.712609
Name: data2, dtype: float64

In [11]:
grouped.get_group('a')['data2']

0   -0.946989
1   -0.649236
4   -0.712609
Name: data2, dtype: float64

### Group by external keys

In [12]:
#define external key years as numpy array
years = np.array([2005, 2005, 2006, 2005, 2006])
df['data1'].groupby([years]).mean()


2005    0.027319
2006   -0.169356
Name: data1, dtype: float64

### Group by functions

In [13]:
#sort by column and retun top n
def top(df, n=5, column='data1'):
    return df.sort_values(by=column)[-n:]

df.groupby(df['key1']).apply(top,n=5)

key1 key2     data1     data2
key1                                
a    4    a  one -0.934310 -0.712609
     0    a  one -0.317571 -0.946989
     1    a  two -0.315828 -0.649236
b    2    b  one  0.595599 -1.119620
     3    b  two  0.715355 -0.160160

### Group-wise aggregation

#### Typical build in aggregation functions:
* sum
* mean
* max / min
* quantile
* ...

In [16]:
#aggregate over the groups
grouped.mean()

,data1,data2
key1,,
a,-0.522570,-0.769611
b,0.655477,-0.639890


#### Custom Aggregation Functions

In [17]:
def peak_to_peak(arr):
    return arr.max() - arr.min()
grouped.agg(peak_to_peak)

,data1,data2
key1,,
a,0.618482,0.297753
b,0.119756,0.959460


#### Multiple aggregations

In [18]:
#just call a list of function
grouped.agg([peak_to_peak, 'mean', 'median'])

data1                            data2                    
     peak_to_peak      mean    median peak_to_peak      mean    median
key1                                                                  
a        0.618482 -0.522570 -0.317571     0.297753 -0.769611 -0.712609
b        0.119756  0.655477  0.655477     0.959460 -0.639890 -0.639890

#### Suppressing the Group Keys 

In [19]:
df.groupby(df['key1']).apply(top,n=2)

key1 key2     data1     data2
key1                                
a    0    a  one -0.317571 -0.946989
     1    a  two -0.315828 -0.649236
b    2    b  one  0.595599 -1.119620
     3    b  two  0.715355 -0.160160

In [20]:
df.groupby(df['key1'], group_keys=False).apply(top,n=2)

,key1,key2,data1,data2
0,a,one,-0.317571,-0.946989
1,a,two,-0.315828,-0.649236
2,b,one,0.595599,-1.119620
3,b,two,0.715355,-0.160160


## More  Exercises in the Lab session... 